In [1]:
import numpy as np

In [2]:
nnArchitecture = [
    {'inputDim':2,'outputDim':25,'activation':'relu'},
    {'inputDim':25,'outputDim':50,'activation':'relu'},
    {'inputDim':50,'outputDim':50,'activation':'relu'},
    {'inputDim':50,'outputDim':25,'activation':'relu'},
    {'inputDim':25,'outputDim':1,'activation':'sigmoid'}
]

In [3]:
def initializeLayer (nnArchitecture,seed=5) :
    np.random.seed(seed)
    paramsValues = {}
    
    for idx,layerInfo in enumerate(nnArchitecture):
        layerNum = idx+1
        inputDim = layerInfo['inputDim']
        outputDim = layerInfo['outputDim']
        
        paramsValues ['W_' + str(layerNum)] = np.random.randn(outputDim,inputDim)*.1
        paramsValues ['B_' + str(layerNum)] = np.random.randn(outputDim,1)*.1
        
    return paramsValues

In [5]:
def sigmoid(Z):
    return 1/(1+np.exp(-Z))

def relu(Z):
    return np.maximum(0,Z)

def sigmoidBackward(dA, Z):
    sig = sigmoid(Z)
    return dA * sig * (1 - sig)

def reluBackward(dA, Z):
    dZ = np.array(dA, copy = True)
    dZ[Z <= 0] = 0;
    return dZ

In [6]:
def singleLayerForwordPropagation (weightMatrixOfCurrentLayer,biasOfCurrentLayer,outputValueOfPreviousLayer,activation = 'relu' ) :
    
    affineTransformationOfCurrentLayer = np.dot(weightMatrixOfCurrentLayer,
                                    outputValueOfPreviousLayer) + biasOfCurrentLayer
    
    if activation is "relu":
        activation_func = relu
    elif activation is "sigmoid":
        activation_func = sigmoid
    else:
        raise Exception('Non-supported activation function')
        
    return activation_func(affineTransformationOfCurrentLayer), affineTransformationOfCurrentLayer

In [7]:
def fullForwordPropagation(inputLayer,nnArchitecture,layerWeightBias) :
    
    outputOfCurrentLayer = inputLayer # input layer
    forwordMemory = {}
    
    for layerIdx,layerInfo in enumerate(nnArchitecture):
        
        currentLayerNum = layerIdx+1

        currentLayerWeight = layerWeightBias['W_'+str(currentLayerNum)]
        currentLayerBias = layerWeightBias['B_'+str(currentLayerNum)]
        currentLayerActivation = layerInfo['activation']
        inputOfCurrentLayer = outputOfCurrentLayer

        outputOfCurrentLayer,affineTransformationOfCurrentLayer = singleLayerForwordPropagation(currentLayerWeight,currentLayerBias,inputOfCurrentLayer,currentLayerActivation)
        

        forwordMemory['A_'+str(layerIdx)] = inputOfCurrentLayer
        forwordMemory['Z_'+str(currentLayerNum)] = affineTransformationOfCurrentLayer
        
    return outputOfCurrentLayer , forwordMemory

In [8]:
def singleLayerBackwordPropagation (weightMatrixOfCurrentLayer,backwordInputValueOfCurrentLayer
                                    ,forwordAffineTransformationOfCurrentLayer,forwordOutputOfpreviousLayer,activation = 'relu' ) :
    
    m = forwordOutputOfpreviousLayer.shape[1]
    
    if activation is "relu":
        activation_func = relu_backward
    elif activation is "sigmoid":
        activation_func = sigmoid_backward
    else:
        raise Exception('Non-supported activation function')

    backwordAffineTransformationOfCurrentLayer = activation_func(backwordInputValueOfCurrentLayer,forwordAffineTransformationOfCurrentLayer)

    backwordOutputOfCurrentLayer = np.dot(weightMatrixOfCurrentLayer.T,backwordAffineTransformationOfCurrentLayer)
    backwordBiasOfCurrentLayer = np.sum(backwordAffineTransformationOfCurrentLayer, axis=1, keepdims=True)/m
    backwordWeightOfCurrentLayer = np.dot(backwordAffineTransformationOfCurrentLayer,forwordOutputOfpreviousLayer.T)/m
    

        
    return backwordOutputOfCurrentLayer,backwordWeightOfCurrentLayer,backwordBiasOfCurrentLayer

In [9]:
def fullBackwordPropagation(forwordOutput,actualValue,nnArchitecture,layerWeightBias,forwordMemory) :
    
    actualValue = actualValue.reshape(forwordOutput.shape)
    

    backwordOutputOfCurrentLayer = - (np.divide(actualValue, forwordOutput) - np.divide(1 - actualValue, 1 - forwordOutput))


    backwordWeightBias = {}
    actualValue = actualValue.reshape(forwordOutput.shape[1])
    for previousLayerNum,layerInfo in reversed(list(enumerate(nnArchitecture))):

        currentLayerNum = previousLayerNum+1
        

        
        currentLayerWeight = layerWeightBias['W_'+str(currentLayerNum)]
        forwordOutputOfpreviousLayer = forwordMemory['A_'+str(previousLayerNum)]

        
        forwordAffineTransformationOfCurrentLayer = forwordMemory['Z_'+str(currentLayerNum)]
        currentLayerActivation = layerInfo['activation']
        
        backwordInputOfCurrentLayer = backwordOutputOfCurrentLayer
        
        backwordOutputOfCurrentLayer,backwordWeight,backwordBias = singleLayerBackwordPropagation(currentLayerWeight
            ,backwordInputOfCurrentLayer ,forwordAffineTransformationOfCurrentLayer,forwordOutputOfpreviousLayer
                                                                ,currentLayerActivation)
        

        backwordWeightBias['W_'+str(currentLayerNum)] = backwordWeight
        backwordWeightBias['B_'+str(currentLayerNum)] = backwordBias
        
    return   backwordWeightBias

In [10]:
def layerWeightBiasUpdate (layerWeightBias,backwordWeightBias,nnArchitecture,learningrate):
    
    for currentLayerNum,layerInfo in enumerate(nnArchitecture,1):

        layerWeightBias['W_'+str(currentLayerNum)] -= learningrate * backwordWeightBias['W_'+str(currentLayerNum)]
        
        layerWeightBias['B_'+str(currentLayerNum)] -= learningrate * backwordWeightBias['B_'+str(currentLayerNum)]

        
    return layerWeightBias